### VERGE: vecotr-mode Regional Geospatial Embeddings
# Initialize Region Of Interest

In this project we will analyze a particular region of interest, defined by a
lon/lat bounding box. Here we save a dict with all ROI-specific parameters.

In [ ]:
import pyproj
import numpy as np
import json
import os

In [ ]:
# ROI name
roi_name = 'ne-dev' # New England, for development

# ROI bounding box.
lat0, lon0 = 41.253746, -73.564321
lat1, lon1 = 45.116468, -68.058533

# Size of tiles, in meters.
tile_size = 2000 

# Resolution of encodings, in meters.
encoding_resolution = 100

# How much to shift consecutive tiles when tiling an AOI, in meters.
tile_shift = 1000

In [ ]:
# The top-level directory for this project.
project_home = '..'

# The name of the ROI to use.
roi_name = 'ne-dev'

# The name of the general-purpose data directory.
data_home = '%s/data' % (project_home)
os.makedirs(data_home, exist_ok=True)

# The name of the ROI-specific data directory.
roi_home = '%s/data/%s' % (project_home, roi_name)
os.makedirs(roi_home, exist_ok=True)


In [ ]:
# Get a good map projection for this ROI.
center_lat = (lat0 + lat1) / 2
center_lon = (lon0 + lon1) / 2

# Local transverse mercator projection.
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
+k=1.0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs
"""
ltm_crs = pyproj.CRS.from_proj4(proj_def)

# Forward and inverse transforms with WGS84.
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

# Get the coords of the lower left corner.
x0, y0 = proj_forward(lon0, lat0)

# Get an offset.
x_offset = -np.floor(x0 / 1000) * 1000
y_offset = -np.floor(y0 / 1000) * 1000


In [ ]:
# Re-define the local transverse mercator projection, with offsets.
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon}
+k=1.0 +x_0={x_offset} +y_0={y_offset} +datum=WGS84 +units=m +no_defs
"""
ltm_crs = pyproj.CRS.from_proj4(proj_def)

# Forward and inverse transforms with WGS84.
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform

# Get the coords of the lower left corner.
x0, y0 = proj_forward(lon0, lat0)
print(x0, y0)


In [ ]:
print(proj_def)

In [ ]:
# Save all that.
import json
roi = {
    'name': roi_name,
    'lon0': lon0,
    'lat0': lat0,
    'lon1': lon1,
    'lat1': lat1,
    'proj_def': proj_def,
    'tile_size': tile_size,
    'tile_shift': tile_shift,
    'encoding_resolution': encoding_resolution
}

fname = '%s/roi.json' % roi_home
with open(fname, 'w') as dest:
    json.dump(roi, dest, indent=4)
